# Reddit sentiment <a class="anchor" id="top"></a>

## TOC:
* [Reddit filters and sentiment methods](#bullet1)
* [General functions and imports](#bullet2)
    - [Filter methods](#sub-bullet2.1)
* [Reddit method 1](#bullet3)
* [Reddit method 2](#bullet4)
* [Sentiment calculations](#bullet5)
    - [VADER sentiment](#sub-bullet5.1)
    - [finBERT sentiment](#sub-bullet5.2)
    - [Relative volume](#sub-bullet5.3)
* [ToDo](#ToDo)

## Reddit filters and sentiment methods <a class="anchor" id="bullet1"></a>

Reddit sentiment is calculated in different ways. I use 2 different 'Reddit-methods' and 2 different [relevancy filters](#sub-bullet2.1) to approach the Reddit data. 

| Filter | Method 1 | Method 2 |
| --- | --- | --- |
| Filter 1 | m1f1 | m2f1 |
| Filter 2 | m1f2 | m2f2 |


On these four combinations, two different sentiment analysis methods will be performed.


**Reddit methods**
- The first way is similar to the method used for Twitter posts. All Reddit comments will be scanned and filtered for stock mentions. These stock mentions are then counted for each stock and subsequent sentiment analysis will be performed on them.
- The second method will not only look at comments, but also consider the Reddit posts. This is done in line with [Hu, Jones, Zhang and Zhang (2021) page 10-12](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3807655). The method begins start by checking whether a post or comment contains a company ticker or name. Once such a mention is found, any direct reply's are searched. All posts or comments mentioning a company together with any direct reply's to these posts or comments are counted as social media activity for this particular company. 

These methods differ slightly from the Reddit comments. The reason for this is that it seems that the `$ticker` naming convention does not seem popular on Reddit. Searching only for `$tsla` returns only 58 observations for the r/stocks subreddit during the 2020_07 period. When searching for the company name instead (Tesla), 4192 observations are found. Hence Reddit comments will also be searched using company names.


A total of `20.511.418` comments are scraped from the 5 subreddits investing, pennystocks, stockmarket, stocks and wallstreetbets. `844.715` comments posts are found for method 1. Method 2 ... `36.244` posts

**Sentiment methods**
The sentiment methods which are used are:
- cjhutto VADER sentiment
- finBERT sentiment



## General functions and imports <a class="anchor" id="bullet2"></a>

[Go back up](#top)

**Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# import statsmodels.formula.api as sm


**General functions**

In [2]:
# List of company names
company_names = {'TSLA': 'Tesla', 
                 'MU': 'Micron Technology', 
                 'SNAP': 'Snapchat', 
                 'AMD': 'AMD', 
                 'DIS': 'Disney', 
                 'MSFT': 'Microsoft', 
                 'AAPL': 'Apple', 
                 'AMZN': 'Amazon', 
                 'SQ': 'Block', 
                 'BABA': 'Alibaba', 
                 'V': 'Visa', 
                 'NFLX': 'Netflix', 
                 'IQ': 'IQIYI', 
                 'ATVI': 'Activision Blizzard', 
                 'SHOP': 'Shopify', 
                 'BA': 'Boeing', 
                 'NVDA': 'NVIDIA', 
                 'GE': 'General Electric', 
                 'WMT': 'Walmart', 
                 'SBUX': 'Starbucks', 
                 'F': 'Ford', 
                 'TLRY': 'Tilray', 
                 'LULU': 'Lululemon', 
                 'BAC': 'Bank of America', 
                 'GME': 'GameStop'}

# Return name of company name dict
def find_company_name(ticker):
    ticker = ticker.upper()
    
    return company_names[ticker]
    
    
find_company_name("gme")

'GameStop'

In [97]:
# A list of top NASDAQ and S&P companies
company_list = ['MetLife', 'Exelon', "O'Reilly Automotive", 'Baker Hughes', 'Rivian',
                'Applied Materials', 'Palo Alto Networks', 'Alphabet', 'Nike',
                'ASML', 'Lululemon', 'Bank of America', 'Cadence Design Systems',
                'BNY Mellon', 'Pfizer', 'Cintas', 'Google', 'Marriott International', 'American Tower',
                'Thermo Fisher Scientific', 'CrowdStrike', 'Ross Stores', 'Emerson', 'Lilly', 'Linde',
                'Salesforce', 'Qualcomm', 'CoStar Group', 'T-Mobile','TMobile', 'Caterpillar', 'Cognizant',
                'FedEx', 'JD.com', 'Johnson & Johnson', 'Alibaba', 'Netflix', 'Seagen', 'Sirius XM',
                'Procter & Gamble', 'Microchip Technology', 'PepsiCo', 'Nvidia', 'Dow', 'Zscaler',
                'Abbott', 'Charter Communications', 'Charles Schwab', "McDonald's", 'Mastercard',
                'Ford', 'MercadoLibre', 'Booking Holdings', 'Diamondback Energy', 'Dollar Tree',
                'Verisk', 'Zoom Video Communications', 'Altria', 'IQIYI', 'Constellation Energy',
                'Wells Fargo', 'Starbucks', 'NXP', 'Adobe', 'eBay', 'Gilead', 'American Electric Power',
                'Lucid Motors', 'Disney', 'Coca-Cola', 'Visa', 'Illumina, Inc.', 'Moderna', 'Fiserv',
                'Boeing', 'Medtronic', 'Gilead Sciences', 'GlobalFoundries', 'NextEra Energy',
                'Texas Instruments', 'Intel', 'Monster Beverage', 'Block', 'Meta Platforms', 'Tesla',
                'KLA Corporation', 'Broadcom', 'Airbnb', 'PayPal', 'CVS Health', 'AT&T', 'Cisco',
                'Raytheon Technologies', 'U.S. Bank', 'Duke Energy', 'Union Pacific', '3M',
                'Lockheed Martin', 'Ansys', 'ExxonMobil', 'Verizon', 'Xcel Energy', 'Paccar', 'Costco',
                'Analog Devices', 'AbbVie', 'Lam Research', 'Vertex Pharmaceuticals', 'Intuitive Surgical',
                'Synopsys', 'AMD', 'American Express', 'Regeneron', 'Activision Blizzard', 'Target',
                'Atlassian', 'Advanced Micro Devices', 'Warner Bros. Discovery', 'Tilray', 'BlackRock',
                'Amazon', 'Walmart', 'Workday', 'Dr Pepper', 'JPMorgan Chase', 'Meta', 'Copart',
                'Biogen', 'Paychex', 'Capital One', 'UnitedHealth Group', 'Fastenal', 'Microsoft',
                'AstraZeneca', "Lowe's", 'Datadog', 'General Electric', 'ADP', 'Goldman Sachs', 'PDD Holdings',
                'Accenture', 'Mondelēz International', 'United Parcel Service', 'Comcast', 'Micron Technology',
                'CSX Corporation', 'Electronic Arts', 'Marvell Technology', 'ConocoPhillips',
                'Walgreens Boots Alliance', 'Home Depot', 'Idexx Laboratories', 'IBM', 'Oracle', 'Shopify',
                'Apple', 'GE', 'Morgan Stanley', 'Merck', 'Citigroup', 'Simon', 'Amgen',
                'Philip Morris International', 'General Dynamics', 'Bristol Myers Squibb', 
                'Old Dominion Freight Line', 'Snapchat', 'GameStop', 'Align Technology', 'Colgate-Palmolive', 
                'Berkshire Hathaway', 'Enphase Energy', 'Fortinet', 'Intuit', 'Danaher', 'Kraft Heinz', 'Chevron', 
                'GM', 'DexCom', 'Southern Company', 'Autodesk', 'American International Group', 'Honeywell']


In [5]:
pattern = re.compile('|'.join(['|'.join([f"\\b{name}\\b" for name in company_list])]))
pattern

re.compile(r"\bMetLife\b|\bExelon\b|\bO'Reilly Automotive\b|\bBaker Hughes\b|\bRivian\b|\bApplied Materials\b|\bPalo Alto Networks\b|\bAlphabet\b|\bNike\b|\bASML\b|\bLululemon\b|\bBank of America\b|\bCadence Design Systems\b|\bBNY Mellon\b|\bPfizer\b|\bCintas\b|\bGoogle\b|\bMarriott International\b|\bAmerican Tower\b|\bThermo Fisher Scientific\b|\bCrowdStrike\b|\bRoss Stores\b|\bEmerson\b|\bLilly\b|\bLinde\b|\bSalesforce\b|\bQualcomm\b|\bCoStar Group\b|\bT-Mobile\b|\bTMobile\b|\bCaterpillar\b|\bCognizant\b|\bFedEx\b|\bJD.com\b|\bJohnson & Johnson\b|\bAlibaba\b|\bNetflix\b|\bSeagen\b|\bSirius XM\b|\bProcter & Gamble\b|\bMicrochip Technology\b|\bPepsiCo\b|\bNvidia\b|\bDow\b|\bZscaler\b|\bAbbott\b|\bCharter Communications\b|\bCharles Schwab\b|\bMcDonald's\b|\bMastercard\b|\bFord\b|\bMercadoLibre\b|\bBooking Holdings\b|\bDiamondback Energy\b|\bDollar Tree\b|\bVerisk\b|\bZoom Video Communications\b|\bAltria\b|\bIQIYI\b|\bConstellation Energy\b|\bWells Fargo\b|\bStarbucks\b|\bNXP\b|\bAdobe\b

### Filter methods <a class="anchor" id="sub-bullet2.1"></a>

Besides the two [Reddit-methods](#bullet1) which were described in the introduction, I also make use of two different filter. These filters are used to check if a Tweet is relevant to a specific company.

To summarize:
- `filter_data_1` Checks whether the name of the company or the ticker of the company is mentioned. If one of these is true, the comment or post is seen as relevant.
- `filter_data_2` Is more strict that filter_data_1. Just like the filter 1, is checks for the company name and ticker. However, this time it also checks whether other tickers or company names are mentioned. It does this using NASDAQ and S&P company names which are summarized in `company_list`. These company names are then compiled into a regex search. If it turns out that different tickers or company names are mentioned in the post or comment, the post or comment is not considered relevant. If only the company ticker or company's name is mentioned, the post <u>is</u> considered relevant.

In [4]:
# Functions borrowed from Twitter 'Data cleaning and sentiment' Jupyter Notebook
def clean_text(text):
    # Remove twitter Return handles (RT @xxx:)
    text = re.sub("RT @[\w]*:", "", text)

    # Remove twitter handles (@xxx)
    text = re.sub("@[\w]*", "", text)

    # Remove URL links (httpxxx)
    url_matcher = "((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"
    text = re.sub(url_matcher, "", text)
    
    # Remove any multiple white spaces, tabs or newlines
    text = re.sub('\s+',' ', text)
    
    #remove “”
    text = re.sub("“|”", "", text)
    
    return text

# This function is slightly adjusted. Not only are company tickers searched ("$ticker"), but also company names are searched.
# This change increases the amount of mentions for Tesla on the r/stocks subreddit during the 2020_07 period from 58 to 4192 observations
def filter_data_1(post, ticker):
    # Filter out posts that do not mention the company ticker.
    company_name = find_company_name(ticker)
    if bool(re.search(fr"(\${ticker})|({company_name})", post, re.IGNORECASE)):
        return True
    else:
        return False
    
# Method 2 filters the comments based on the rule that exactly 1 ticker or company name is mentioned and ...
# ... that this ticker is the ticker of the company of which the sentiment is being calculated   
def filter_data_2(post, ticker):
    # ---- Tickers ----
    # Count the number of tickers in the post
    ticker_matches = len(re.findall(r"\$[a-zA-Z]+", post, re.IGNORECASE))
    company_ticker_matches = len(re.findall(fr"\${ticker}", post, re.IGNORECASE))
    
    # The ticker_diff needs to be equal to zero, else other tickers than the company ticker are mentioned.
    ticker_diff = ticker_matches - company_ticker_matches
    
    # print(f"{len(ticker_matches)} - {len(company_ticker_matches)} = {ticker_diff}")
    
    # ---- Company name ----
    # Create pattern which matches any of the company names in the company_list
    # '\\b' prevents any occurences of short company names from being picked up mid-string:
    # GE --> 'Vegetable' would be picked up without this rule
    pattern = re.compile('|'.join(['|'.join([f"\\b{name}\\b" for name in company_list])]))
    name_matches = len(pattern.findall(post, re.IGNORECASE))
    
    # Next the company of the ticker is searched.
    company_name = find_company_name(ticker)
    company_name_matches = len(re.findall(f"\\b{company_name}\\b", post, re.IGNORECASE))
    
    # The company_name_diff needs to be equal to zero, else other tickers than the company ticker are mentioned.
    company_name_diff = name_matches - company_name_matches
    
    # print(f"{name_matches_count} - {len(company_name_matches)} = {company_name_diff}")
    
    # Filter out posts with more or less than 1 ticker, and check whether this 1 ticker is the company ticker.
    if ticker_diff == 0 and company_name_diff == 0 and (company_ticker_matches > 0 or company_name_matches > 0):
        return True
    else:
        return False

text = "This is a test text. If Apple is in the text, without any other companies mentioned, the functions returns True."
print(filter_data_2(text, "aapl"))

text = "If another company is mentioned, like Gamestop, it returns False"
print(filter_data_2(text, "aapl"))

text = "If no company is mentioned, it also returns False"
print(filter_data_2(text, "aapl"))

True
False
False


In [8]:
def filter_mentions(df, ticker, social_type='comment', filter_type=1):
    if social_type == 'comment':
        # Dropping any potential NA's
        df = df[df['body'].notna()]

        if filter_type == 1:
            # Filter out comments that do not mention company ticker using 'filter_data_1'
            df_filter = df['body'].apply(filter_data_1, ticker=ticker)
        elif filter_type == 2:
            # Filter out comments that do not mention company ticker using 'filter_data_2'
            df_filter = df['body'].apply(filter_data_2, ticker=ticker)
        else:
            raise Exception("Please select a valid filter_type for func [filter_mentions]. Either 1 or 2.")
        df = df[df_filter]
        
        # Skip cleaning if df is empty
        if df.shape[0] > 0:
            # Clean text
            df['body'] = df['body'].apply(clean_text)  
            
    elif social_type == 'post':
        # Dropping any potential NA's
        df = df[df['selftext'].notna()]

        if filter_type == 1:
            # Filter out posts that do not mention company ticker using 'filter_data_1'
            selftext_filter = df['selftext'].apply(filter_data_1, ticker=ticker)
            title_filter = df['title'].apply(filter_data_1, ticker=ticker)
        elif filter_type == 2:
            # Filter out posts that do not mention company ticker using 'filter_data_2'
            selftext_filter = df['selftext'].apply(filter_data_2, ticker=ticker)
            title_filter = df['title'].apply(filter_data_2, ticker=ticker)
        else:
            raise Exception("Please select a valid filter_type for func [filter_mentions]. Either 1 or 2.")
        df = df[selftext_filter | title_filter]
        
        
        # Skip cleaning if df is empty
        if df.shape[0] > 0:
            # Clean text
            df['selftext'] = df['selftext'].apply(clean_text)
           
        # For posts specifically, I drop observations for
        # - Authors postings multiple posts on 1 day (spam or duped posts)
        df = df.drop_duplicates(subset=['author', 'date'], keep=False)
        # - Posts with duplicate texts
        df = df.drop_duplicates(subset=['selftext'], keep=False)
        
    else:
        raise Exception("Please select a valid social_type for func [filter_mentions]. Either 'comment' or 'post'.")
    
    return df

def save_df(df, save_path):
    # Check if file already exists. Ignore headers if true
    if os.path.isfile(save_path):
        df.to_csv(save_path, mode='a', header=False, index=False)
    else: 
        df.to_csv(save_path, encoding='utf-8', index=False)
        print(f"Creating new file at [{save_path}]")
        

## Reddit method 1 <a class="anchor" id="bullet3"></a>

**Extract all comments which mention a company ticker or name**

In [40]:
save = False

start_time = time.time()

ticker_list = ['AAPL', 'AMD', 'AMZN', 'ATVI', 'BA', 'BABA', 'BAC', 'DIS', 'F', 'GE', 'GME', 'IQ', 'LULU', 'MSFT', 'MU', 'NFLX', 'NVDA', 'SBUX', 'SHOP', 'SNAP', 'SQ', 'TLRY', 'TSLA', 'V', 'WMT']
rootdir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\unfiltered"
save_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\filtered"

if save:
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            # Create csv_path
            csv_path = os.path.join(subdir, file)


            # Read csv
            df = pd.read_csv(csv_path)

            # Dropping certain columns
            df.drop(columns=['score_hidden', 'total_awards_received', 'nest_level', 'author_fullname'], inplace=True)
            
            # Loop tickers
            for ticker in ticker_list:
                print(f"Time passed: {str(round((time.time() - start_time), 1)).ljust(8)} - Processing {ticker.ljust(4)} for [{csv_path}]", end='\r')

                # ----------------------- Filter_1 -----------------------
                work_df = df.copy()
                # Apply filter
                filtered_df = filter_mentions(work_df, ticker, social_type='comment', filter_type=1)

                # Creating save_path and saving file (either new file or appending)
                save_path = os.path.join(save_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')
                save_df(filtered_df, save_path)

                # ----------------------- Filter_2 -----------------------
                work_df = df.copy()
                # Apply filter
                filtered_df = filter_mentions(work_df, ticker, social_type='comment', filter_type=2)

                # Creating save_path and saving file (either new file or appending)
                save_path = os.path.join(save_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')
                save_df(filtered_df, save_path)

Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/comments/filtered/filter_2/AAPL.csv]ltered\investing\2018_04.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/comments/filtered/filter_2/AMD.csv]iltered\investing\2018_04.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/comments/filtered/filter_2/AMZN.csv]ltered\investing\2018_04.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/comments/filtered/filter_2/ATVI.csv]ltered\investing\2018_04.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/comments/filtered/filter_2/BA.csv]filtered\investing\2018_04.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/comments/filtered/filter_2/BABA.csv]ltered\investing\2018_04.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/comments/filtered/filter_2/BAC.csv]iltered\investing\2018_04.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/r

## Reddit method 2 <a class="anchor" id="bullet4"></a>

Method 2 checks what posts contain mentions of companies in the form of ticker or company name mentions. 

In [9]:
save = False

ticker_list = ['AAPL', 'AMD', 'AMZN', 'ATVI', 'BA', 'BABA', 'BAC', 'DIS', 'F', 'GE', 'GME', 'IQ', 'LULU', 'MSFT', 'MU', 'NFLX', 'NVDA', 'SBUX', 'SHOP', 'SNAP', 'SQ', 'TLRY', 'TSLA', 'V', 'WMT']
rootdir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\posts\unfiltered"
save_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\posts\filtered"

if save:
    start_time = time.time()
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            # Create csv_path
            csv_path = os.path.join(subdir, file)


            # Read csv
            df = pd.read_csv(csv_path)

            # Loop tickers
            for ticker in ticker_list:
                print(f"Time passed: {str(round((time.time() - start_time), 1)).ljust(8)} - Processing {ticker.ljust(4)} for [{csv_path}]", end='\r')
                # ----------------------- Filter_1 -----------------------
                work_df = df.copy()
                # Apply filter
                filtered_df = filter_mentions(df, ticker, social_type='post', filter_type=1)


                # Creating save_path and saving file (either new file or appending)
                save_path = os.path.join(save_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')
                save_df(filtered_df, save_path)

                # ----------------------- Filter_2 -----------------------
                work_df = df.copy()
                # Apply filter
                filtered_df = filter_mentions(work_df, ticker, social_type='post', filter_type=2)

                # Creating save_path and saving file (either new file or appending)
                save_path = os.path.join(save_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')
                save_df(filtered_df, save_path)


Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_1/AAPL.csv]ltered\filtered_df.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_2/AAPL.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_1/AMD.csv]iltered\filtered_df.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_2/AMD.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_1/AMZN.csv]ltered\filtered_df.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_2/AMZN.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_1/ATVI.csv]ltered\filtered_df.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_2/ATVI.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/posts/filtered/filter_1/B

**Merging all comments to a single file**

After finding all posts mentioning a company (besides the comments as done under method 1), it is now time to see what replies are posted to these comments. For posts these replies will be top level comments. For replies on other comments, they can be found on any level. 

I begin by merging all comments into a single file. This does not serve any purpose besides making it easier to loop the comments.

In [5]:
save = False

# --- Merge all comments into single file ---
if save:
    rootdir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\unfiltered"
    save_path = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\all_comments.csv"
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            # Create csv_path
            csv_path = os.path.join(subdir, file)

            # Read csv
            df = pd.read_csv(csv_path)

            df.drop(columns=['score_hidden', 'total_awards_received', 'nest_level', 'author_fullname'], inplace=True)

            # Save new csv or append to existing csv
            save_df(df, save_path)

Creating new file at [E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\all_comments.csv]


**Searching for replies**

Method 2 also finds all comments which are replies to posts or comments containing the company name or ticker. 

To find out whether a comment is a reply, each comment will be checked to see if the `parent_id` matches an `id` of a comment containing a company mention. For responses to posts, the `parent_id` will be empty. For these, the `link_id` will need to match the post's id.

For replies to comments:
- `df['parent_id']` matches an `df['id']` of a comment containing a company mention.

For top-level replies to posts:
- `df['parent_id']` is empty and `df['link_id']` is equal to the post's `df['id']` which mentions a company.

In [24]:
def filter_replies(df_chunk, comments_df, posts_df):
    #           --- Comment filter ---
    # Create a dataframe filter which checks if comment is a reply to other comment
    comment_id_list = comments_df['id'].to_list()
    comment_filter = df_chunk['parent_id'].isin(comment_id_list)
    
    #           --- Post filter ---
    # Removing 't3_' from the comment 'link_id' field
    df_chunk['link_id'] = df_chunk['link_id'].str[3:]
    
    # Create a dataframe filter which checks if comment is a reply to other comment
    post_id_list = posts_df['id'].to_list()
    post_filter = (df_chunk['parent_id'].isna() & df_chunk['link_id'].isin(post_id_list))
    
    return_df = df_chunk[comment_filter | post_filter]
    return return_df

# Checks replies for the chunk dataframe input
def check_replies(df):
    
    ticker_list = ['AAPL', 'AMD', 'AMZN', 'ATVI', 'BA', 'BABA', 'BAC', 'DIS', 'F', 'GE', 'GME', 'IQ', 'LULU', 'MSFT', 'MU', 'NFLX', 'NVDA', 'SBUX', 'SHOP', 'SNAP', 'SQ', 'TLRY', 'TSLA', 'V', 'WMT']
    comment_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\filtered"
    posts_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\posts\filtered"
    save_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\replies"
    
    for ticker in ticker_list:
        # ----------------------- Filter_1 -----------------------
        work_df = df.copy()
        # Creating paths
        comment_path = os.path.join(comment_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')
        posts_path = os.path.join(posts_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')
        
        # Reading csv's to df
        comments_df = pd.read_csv(comment_path)
        posts_df = pd.read_csv(posts_path)
        
        # Run function
        return_df = filter_replies(work_df, comments_df, posts_df)

        # Creating save_path and saving file (either new file or appending)
        save_path = os.path.join(save_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')
        save_df(return_df, save_path)

        # ----------------------- Filter_2 -----------------------
        work_df = df.copy()
        # Creating paths
        comment_path = os.path.join(comment_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')
        posts_path = os.path.join(posts_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')

        # Reading csv's to df
        comments_df = pd.read_csv(comment_path)
        posts_df = pd.read_csv(posts_path)
        
        # Run function
        return_df = filter_replies(work_df, comments_df, posts_df)

        # Creating save_path and saving file (either new file or appending)
        save_path = os.path.join(save_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')
        save_df(return_df, save_path)


**Search for replies by chunking all_comments.csv**


To loop all comments, I loop the chunks from the all_comments.csv

In [25]:
save = False

start_time = time.time()

csv_path = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\all_comments.csv"
chunksize = 10 ** 6
counter = 1

if save:
    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        print(f"Processing chunk {str(counter).ljust(2)} --- time passed: {round((time.time() - start_time), 1)}", end='\r')
        check_replies(chunk)
        counter += 1

Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_1/AAPL.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_2/AAPL.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_1/AMD.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_2/AMD.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_1/AMZN.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_2/AMZN.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_1/ATVI.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_2/ATVI.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_1/BA.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Thesis/reddit/replies/filter_2/BA.csv]
Creating new file at [E:/Users/Christiaan/Large_Files/Th

**Merging comments with replies**

Besides this, duplicates are also removed

In [95]:
save = False

ticker_list = ['AAPL', 'AMD', 'AMZN', 'ATVI', 'BA', 'BABA', 'BAC', 'DIS', 'F', 'GE', 'GME', 'IQ', 'LULU', 'MSFT', 'MU', 'NFLX', 'NVDA', 'SBUX', 'SHOP', 'SNAP', 'SQ', 'TLRY', 'TSLA', 'V', 'WMT']
comment_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\filtered"
reply_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\replies"
save_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\method_2"

if save:
    for ticker in ticker_list:
        # ----------------------- Filter_1 -----------------------
        comment_path = os.path.join(comment_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')
        reply_path = os.path.join(reply_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')
        save_path = os.path.join(save_dir, "filter_1", f"{ticker}.csv").replace('\\', '/')

        # Reading csv's to df
        comments_df = pd.read_csv(comment_path)
        reply_df = pd.read_csv(reply_path)

        # Merge files
        merged_df = pd.concat([comments_df, reply_df], ignore_index=True)

        # Delete duplicates
        merged_df.drop_duplicates(subset=['permalink'], inplace=True)

        # Saving file
        merged_df.to_csv(save_path, encoding='utf-8', index=False)

        # ----------------------- Filter_2 -----------------------
        comment_path = os.path.join(comment_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')
        reply_path = os.path.join(reply_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')
        save_path = os.path.join(save_dir, "filter_2", f"{ticker}.csv").replace('\\', '/')

        # Reading csv's to df
        comments_df = pd.read_csv(comment_path)
        reply_df = pd.read_csv(reply_path)

        # Merge files
        merged_df = pd.concat([comments_df, reply_df], ignore_index=True)

        # Delete duplicates
        merged_df.drop_duplicates(subset=['permalink'], inplace=True) 

        # Saving file
        merged_df.to_csv(save_path, encoding='utf-8', index=False)

## Sentiment calculations <a class="anchor" id="bullet5"></a>

Having the results for all four different 'Reddit-method' - filter combinations, I can now start with sentiment calculations. 

### VADER sentiment <a class="anchor" id="sub-bullet5.1"></a>

**Functions**

These functions are partially copied from the Twitter sentiment Jupyter Notebook.

In [2]:
# Adding word-sentiment pairs to the cjhutto vaderSentiment library.
new_words = {}

# Adding custom postive words
positive_words = {
    'buy': 2.0,
    'buying': 2.0,
    'bullish': 2.0,
    'long': 1.0,
    'call': 1.0,
    'calls': 1.0,
    'rocket': 3.0,        # Added for 'rocket' emoji 🚀
    'increasing': 2.0,     # Added for 'chart increasing' emoji 📈
    'to the moon': 2.5,
    "undervalued": 2.0
}
# Adding custom negative words
negative_words = {
    'decreasing': -2.0,   # Added for 'chart increasing' emoji 📉
    'sell': -2.0,
    'selling': -2.0,
    'bearish': -2.0,
    'put': -1,
    'puts': -1,
    'short': -1.0,
    'shorting': -1.5,
    "overvalued": -2.0,
    'expensive': -1.5
}

# Adding positive and negative words to new_worddictionary
new_words.update(positive_words)
new_words.update(negative_words)

In [3]:
#   ---------------------------   Sentiment   ---------------------------
# Creating SIA, which uses standard words.
SIA = SentimentIntensityAnalyzer()

def calc_sentiment_1(text, sent_type):
    result = SIA.polarity_scores(text)
    return result[sent_type]

# Creating SIA2 to add custom words.
SIA2 = SentimentIntensityAnalyzer()
SIA2.lexicon.update(new_words)

def calc_sentiment_2(text, sent_type):
    result = SIA2.polarity_scores(text)
    return result[sent_type]

**finBERT filter and table deletion** <a class="anchor" id="delete-table-info2"></a>

During the preperation of the next sentiment method, I discover that the finBERT has a maximum limit of 512 words. Unlike the Twitter posts, Reddit comments can actually exceed this amount. Hence I decide to filter the 5% largest post. Besides this, to put as many useful words in the 512 word limit, I delelete tables from user's comments. More info about this can be found [here](#delete-table-info)

In [4]:
# This function removes any tables from reddit comments.
def clean_text(text): 
    # Remove tables
    text = re.sub("[^\|\s]{1,20}\s*(?=\|)|(?<=\|)\s*[^\|]{1,30}\s*(?=\|)|(?<=\|)\s*[^\|\s]{1,30}", "", text)
    text = re.sub("\|", "", text)

    
    return text

In [5]:
def finBERT_filter(df):
    # Dropping any NA in body
    df.dropna(subset=['body'], inplace=True)
        
    # Cleaning tables from posts
    df['body'] = df['body'].apply(clean_text)

    # Getting text lengths  
    df['len'] = df['body'].str.len()

    # Dropping texts above the 95% quantile
    quantile_95 = df['len'].quantile(0.95)
    df = df[df['len'] < quantile_95]
    
    # Resetting index
    df.reset_index(inplace=True)
    
    return df

**Main function**

In [6]:
def clean_data(df, ticker):
    start_time = time.time()
    # Prepping data by setting date column
    df['created_at'] = pd.to_datetime(df['utc_datetime_str'])
    df['date'] = df['created_at'].dt.date
    
    # Perform the finBERT filter (see description above)
    df = finBERT_filter(df).copy()
    
    #   ---------------------------   Sentiment   ---------------------------
    # Calculate sentiment scores
    df[f'compound_sent_1'] = df['body'].astype(str).apply(calc_sentiment_1, sent_type='compound')
    df[f'compound_sent_2'] = df['body'].astype(str).apply(calc_sentiment_2, sent_type='compound')
    
    print(f"[{ticker}] Done calculating sentiment after --- %s seconds ---" % (time.time() - start_time))
    
    """Converting this to pos, neg or neu sentiment
    - positive sentiment: compound score >= 0.05
    - neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
    - negative sentiment: compound score <= -0.05
    """
    
    # Check if sentiment corresponds to pos, neg or neu sentiment for compound_sent_1
    df['s1_pos'] = np.where(df['compound_sent_1'] >= 0.05, 1, 0)
    df['s1_neg'] = np.where(df['compound_sent_1'] <= -0.05, 1, 0)

    # Check if sentiment corresponds to pos, neg or neu sentiment for compound_sent_2
    df['s2_pos'] = np.where(df['compound_sent_2'] >= 0.05, 1, 0)
    df['s2_neg'] = np.where(df['compound_sent_2'] <= -0.05, 1, 0)
   
    return df

In [7]:
def count_posts(df):
    # Create results_df with [sentiment_1]
    results_df = df[['date', 's1_pos', 's1_neg']].groupby('date', as_index=False).sum().rename(columns={"s1_pos": "[s1]pos", "s1_neg": "[s1]neg"})
    results_df['[s1]total'] = results_df['[s1]pos'] + results_df['[s1]neg']

    # Merge [sentiment_2]
    to_merge_df = df[['date', 's2_pos', 's2_neg']].groupby('date', as_index=False).sum().rename(columns={"s2_pos": "[s2]pos", "s2_neg": "[s2]neg"})
    to_merge_df['[s2]total'] = to_merge_df['[s2]pos'] + to_merge_df['[s2]neg']
    results_df = results_df.merge(to_merge_df, how='left', left_on='date', right_on='date')
    
    return results_df

**Different types of VADER sentiment measures** <a class="anchor" id="sentiment-measure"></a>

Although different data filters and sentiment measuring methods are mentioned, this does not solve the problem of how sentiment needs to be measured. Do I take the total amount of positive posts as a proxy of sentiment? Or do I use the ratio of positive and negative posts. Or do I subtract the negative posts from the positive posts and divide that figure by the total amount of posts?

It seems that there is no clear winner here. I propose that sentiment should be measure in two parts. The first part of the measure should contain the actual `sentiment`. Is it positive or negative? It should also capture the severity of the sentiment. The second part should contain the `relative volume` of the social media posts, as large volumes are more likely to be noticable. Part 2 will thus act as a way to strengen or weaken the total sentiment score of the day, by comparing the volume of that day with the average volume of the last 7 days. This part is worked out here: [part 2](#sub-bullet5.3). 

$\text{Sentiment}_{t0} = \text{Sentiment (part 1)}_{t0}*\text{Sentiment relative volume (part 2)}_{t0}$

**Part 1**
The first part is the hardest one, as it is unclear what the best way to measure sentiment is. To tackle this problem, I will calculate the sentiment in different ways.
- Method 1: positive / negative
- <strike>Method 2: (positive - negative) / (positive + negative)</strike>
- Method 3: Daily mean of compound sentiment score, counting only posts categorised as positive or negative


In [9]:
def calc_sent_measures(return_df):
    # Get results dataframe
    sentiment_measures = count_posts(return_df)

    # Only keep sentiment 2 results
    sentiment_measures = sentiment_measures.loc[:, sentiment_measures.columns.str.contains('s1|s2|date')]
    
    # Method 1 - ratio
    sentiment_measures['[s1]method_1'] = sentiment_measures['[s1]pos'] / sentiment_measures['[s1]total']
    sentiment_measures['[s2]method_1'] = sentiment_measures['[s2]pos'] / sentiment_measures['[s2]total']

    # Method 2 - discontinued as it is the same as method 1
    #     sentiment_measures['[f1s2]method_2'] = (sentiment_measures['[f1s2]pos'] - sentiment_measures['[f1s2]neg']) / sentiment_measures['[f1s2]total']
    #     sentiment_measures['[f2s2]method_2'] = (sentiment_measures['[f2s2]pos'] - sentiment_measures['[f2s2]neg']) / sentiment_measures['[f2s2]total']

    # Method 3
    to_merge = return_df[((return_df['s1_pos'] == 1) | (return_df['s1_neg'] == 1))][['date', 'compound_sent_1']].groupby('date').mean().rename(columns={'compound_sent_1': '[s1]method_3'})
    sentiment_measures = sentiment_measures.merge(to_merge, how='left', left_on='date', right_on='date')
    to_merge = return_df[((return_df['s2_pos'] == 1) | (return_df['s2_neg'] == 1))][['date', 'compound_sent_2']].groupby('date').mean().rename(columns={'compound_sent_2': '[s2]method_3'})

    sentiment_measures = sentiment_measures.merge(to_merge, how='left', left_on='date', right_on='date')
    return sentiment_measures


**Actual sentiment calculations**

In [14]:
ticker_list = ['AAPL', 'AMD', 'AMZN', 'ATVI', 'BA', 'BABA', 'BAC', 'DIS', 'F', 'GE', 'GME', 'IQ', 'LULU', 'MSFT', 'MU', 'NFLX', 'NVDA', 'SBUX', 'SHOP', 'SNAP', 'SQ', 'TLRY', 'TSLA', 'V', 'WMT']

save_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\sentiment\VADER"
method_1_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\filtered"
method_2_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\method_2"


for ticker in ticker_list:
    def loop_func(comment_dir, filter_method="filter_1", method_name="m1f1"):
        # ----------------------- m1f1/m1f2/m2f1/m2f2 -----------------------
        # Setting up paths
        comment_path = os.path.join(comment_dir, filter_method, f"{ticker}.csv").replace('\\', '/')
        save_path = os.path.join(save_dir, method_name, f"{ticker}.csv").replace('\\', '/')
        
        # Check if file already exists and skip sentiment calculation if file exists
        if os.path.isfile(save_path):
            print(f"[skipping] File already exists: [{save_path}]")

        else: 
            # Reading csv's to df
            comments_df = pd.read_csv(comment_path)

            # Filter and clean data. Also perform VADER sentiment scoring.
            return_df = clean_data(comments_df, ticker)

            # Calculate sentiment scores for each method
            sentiment_measures = calc_sent_measures(return_df)

            # Save file
            sentiment_measures.to_csv(save_path, encoding='utf-8', index=False)

    # For each of the 4 method-filter combinations, run the the functions
    loop_func(comment_dir=method_1_dir, filter_method="filter_1",  method_name="m1f1")
    loop_func(comment_dir=method_1_dir, filter_method="filter_2",  method_name="m1f2")
    loop_func(comment_dir=method_2_dir, filter_method="filter_1",  method_name="m2f1")
    loop_func(comment_dir=method_2_dir, filter_method="filter_2",  method_name="m2f2")

[AAPL] Done calculating sentiment after --- 82.22627329826355 seconds ---
[AAPL] Done calculating sentiment after --- 27.112491369247437 seconds ---
[AAPL] Done calculating sentiment after --- 106.12035155296326 seconds ---
[AAPL] Done calculating sentiment after --- 40.65776085853577 seconds ---
[AMD] Done calculating sentiment after --- 44.83538103103638 seconds ---
[AMD] Done calculating sentiment after --- 23.831379413604736 seconds ---
[AMD] Done calculating sentiment after --- 64.53612852096558 seconds ---
[AMD] Done calculating sentiment after --- 38.3740599155426 seconds ---
[AMZN] Done calculating sentiment after --- 82.37491154670715 seconds ---
[AMZN] Done calculating sentiment after --- 27.920042753219604 seconds ---
[AMZN] Done calculating sentiment after --- 106.65402173995972 seconds ---
[AMZN] Done calculating sentiment after --- 40.22500658035278 seconds ---
[ATVI] Done calculating sentiment after --- 0.8574912548065186 seconds ---
[ATVI] Done calculating sentiment aft

### finBERT sentiment <a class="anchor" id="sub-bullet5.2"></a>

**Setting up Pytorch and the finBERT model**

In [2]:
import torch


In [3]:
print(torch.__version__)

1.13.1+cu116


In [4]:
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_properties(i).name)


NVIDIA GeForce GTX 750 Ti


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

**Functions**

In [6]:
def calculate_runtime(function):
    """
    A wrapper function that calculates the runtime of the specified function.
    """
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = function(*args, **kwargs)
        end_time = time.time()
        print(f"Runtime for {function.__name__}: {end_time - start_time} seconds")
        return result
    return wrapper

The function below sets up the finBERT data pipeline and initializes the model.

In [7]:
@calculate_runtime
def calc_sent_finbert(df):
    # Inititalise sentiment pipeline
    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer, device=0)

    # Create sentence list and run finBERT
    sentence_list = df['body'].to_list()
    results = nlp(sentence_list)

    # Add results to main dataframe
    results = pd.DataFrame(results)
    df = df.merge(results, how='left', left_index=True, right_index=True)
    return df  #LABEL_0: neutral; LABEL_1: positive; LABEL_2: negative



**Deleted tables from comments** <a class="anchor" id="delete-table-info"></a>

Although the data has already been cleaned, I discover that some posts contain tables. These tables consist out of many small characters and causes the finBERT model to be unable to process the comments. This is due to the 512 word limit the model has. 

This table cleaning is not done in the data seperation filtering part, as it can contain tickers and company names. It is however used in the VADER sentiment, which can be seen [here](#delete-table-info2).

The regex pattern used to filter the tables is quite complex. I would recommend seeing what the regex pattern does using the following [website] (https://regex101.com/) and pasting the regex code together with a text sample.
- regex code: `[^\|\s]{1,20}\s*(?=\|)|(?<=\|)\s*[^\|]{1,30}\s*(?=\|)|(?<=\|)\s*[^\|\s]{1,30}`
- text sample: `"Company | Symbol | Price | Daily Change | 52W Change :-------|:------:|:------:|:------:|:------: General Electric Co | GE | 14.17 | -0.07% | -48.2% Tpi Composites Inc | TPIC | 26.76 | -0.04% | +61.3% [*]( 'Data from IEX')*[13-Week Price Moves](%) - [52 Week Price Change](%) - quote-bot by [echoapollo]()*"`

In [8]:
# Functions borrowed from Twitter 'Data cleaning and sentiment' Jupyter Notebook
def clean_text(text): 
    # Remove tables
    text = re.sub("[^\|\s]{1,20}\s*(?=\|)|(?<=\|)\s*[^\|]{1,30}\s*(?=\|)|(?<=\|)\s*[^\|\s]{1,30}", "", text)
    text = re.sub("\|", "", text)

    
    return text
text = "Company | Symbol | Price | Daily Change | 52W Change :-------|:------:|:------:|:------:|:------: General Electric Co | GE | 14.17 | -0.07% | -48.2% Tpi Composites Inc | TPIC | 26.76 | -0.04% | +61.3% [*]( 'Data from IEX')*[13-Week Price Moves](%) - [52 Week Price Change](%) - quote-bot by [echoapollo]()*"
print(text)
print("\n-----> Turns into: ↓↓↓ \n")
clean_text(text)


Company | Symbol | Price | Daily Change | 52W Change :-------|:------:|:------:|:------:|:------: General Electric Co | GE | 14.17 | -0.07% | -48.2% Tpi Composites Inc | TPIC | 26.76 | -0.04% | +61.3% [*]( 'Data from IEX')*[13-Week Price Moves](%) - [52 Week Price Change](%) - quote-bot by [echoapollo]()*

-----> Turns into: ↓↓↓ 



" [*]( 'Data from IEX')*[13-Week Price Moves](%) - [52 Week Price Change](%) - quote-bot by [echoapollo]()*"

In [9]:
def finBERT_filter(df):
    # Dropping any NA in body
    df.dropna(subset=['body'], inplace=True)
        
    # Cleaning tables from posts
    df['body'] = df['body'].apply(clean_text)

    # Getting text lengths  
    df['len'] = df['body'].str.len()

    # Dropping texts above the 95% quantile
    quantile_95 = df['len'].quantile(0.95)
    df = df[df['len'] < quantile_95]
    
    return df

In [10]:
def clean_data(df_in, ticker):
    df = df_in.copy()
    start_time = time.time()
    # Prepping data by setting date column
    df['created_at'] = pd.to_datetime(df['utc_datetime_str'])
    df['date'] = df['created_at'].dt.date   
    
    # Renaming score to post_score (else conflict with finBERT results)
    df.rename(columns={"score": "post_score"}, inplace=True)
    
    #   ---------------------------   NEW FILTERS   ---------------------------
    # Filters out tables from post and deleted 5% largest posts
    df = finBERT_filter(df)
    
    # Resetting index
    df.reset_index(inplace=True)
    
    #   ---------------------------   Sentiment   ---------------------------
    df = calc_sent_finbert(df)
    print(f"[{ticker}] Done calculating sentiment after --- %s seconds ---" % (time.time() - start_time))
    return df


In [11]:
def count_results(df):
    # Drop scores with less than 80% certainty
    df = df[df['score'] > 0.8]

    # Drop all neutral observations
    df = df[df['label'] != "Neutral"]
    df['pos'] = np.where(df['label'] == "Positive", 1, 0)
    df['neg'] = np.where(df['label'] == "Negative", 1, 0)

    # Create results_df with [filter_1]
    counted_results = df[['date', 'pos', 'neg']].groupby('date', as_index=False).sum().rename(columns={"pos": "[BERT]pos", "neg": "[BERT]neg"})
    counted_results['[BERT]total'] = counted_results['[BERT]pos'] + counted_results['[BERT]neg']

    return counted_results


In [12]:
def calc_sent_measures(return_df):
    # Get results dataframe
    sentiment_measures = count_results(return_df)
    
    # Method 1 - ratio
    sentiment_measures['[BERT]method_1'] = sentiment_measures['[BERT]pos'] / sentiment_measures['[BERT]total']

    # Method 2 - discontinued as it is the same as method 1
    #     sentiment_measures['method_2'] = (sentiment_measures['pos'] - sentiment_measures['neg']) / sentiment_measures['total']

    # Method 3
    # Method 3 does not exist with finBERT

    return sentiment_measures

**Actual loop to calc finBERT sentiment for all files**

In [2]:
ticker_list = ['AAPL', 'AMD', 'AMZN', 'ATVI', 'BA', 'BABA', 'BAC', 'DIS', 'F', 'GE', 'GME', 'IQ', 'LULU', 'MSFT', 'MU', 'NFLX', 'NVDA', 'SBUX', 'SHOP', 'SNAP', 'SQ', 'TLRY', 'TSLA', 'V', 'WMT']

save_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\sentiment\finBERT"
method_1_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\filtered"
method_2_dir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\method_2"


for ticker in ticker_list:
    def loop_func(comment_dir, filter_method="filter_1", method_name="m1f1"):
        # ----------------------- m1f1/m1f2/m2f1/m2f2 -----------------------
        # Creating comment_path and save_path
        comment_path = os.path.join(comment_dir, filter_method, f"{ticker}.csv").replace('\\', '/')
        save_path = os.path.join(save_dir, method_name, f"{ticker}.csv").replace('\\', '/')

        # Check if file already exists and skip sentiment calculation if file exists
        if os.path.isfile(save_path):
            print(f"[skipping] File already exists: [{save_path}]")

        else:
            # Reading csv's to df
            comments_df = pd.read_csv(comment_path)

            # Filter and clean data. Also perform VADER sentiment scoring.
            return_df = clean_data(comments_df, ticker)

            # Calculate sentiment scores for each method
            sentiment_measures = calc_sent_measures(return_df)

            #Save file
            sentiment_measures.to_csv(save_path, encoding='utf-8', index=False)

    # For each of the 4 method-filter combinations, run the the functions
    loop_func(comment_dir=method_1_dir, filter_method="filter_1",  method_name="m1f1")
    loop_func(comment_dir=method_1_dir, filter_method="filter_2",  method_name="m1f2")
    loop_func(comment_dir=method_2_dir, filter_method="filter_1",  method_name="m2f1")
    loop_func(comment_dir=method_2_dir, filter_method="filter_2",  method_name="m2f2")

[skipping] File already exists: [E:/Users/Christiaan/Large_Files/Thesis/reddit/sentiment/finBERT/m1f1/AAPL.csv]
Runtime for calc_sent_finbert: 942.8509826660156 seconds
[AAPL] Done calculating sentiment after --- 945.7433111667633 seconds ---
Runtime for calc_sent_finbert: 4296.0922911167145 seconds
[AAPL] Done calculating sentiment after --- 4308.085411071777 seconds ---
Runtime for calc_sent_finbert: 1720.9407002925873 seconds
[AAPL] Done calculating sentiment after --- 1725.547036409378 seconds ---
Runtime for calc_sent_finbert: 2031.6958684921265 seconds
[AMD] Done calculating sentiment after --- 2036.5930635929108 seconds ---
Runtime for calc_sent_finbert: 1163.1408705711365 seconds
[AMD] Done calculating sentiment after --- 1165.609448671341 seconds ---
Runtime for calc_sent_finbert: 3803.923209667206 seconds
[AMD] Done calculating sentiment after --- 3811.7896823883057 seconds ---
Runtime for calc_sent_finbert: 2280.6406950950623 seconds
[AMD] Done calculating sentiment after --

### Relative volume and final sentiment <a class="anchor" id="sub-bullet5.3"></a>

As explained [here](#sentiment-measure), the sentiment measure consists out of two parts. The second part is now discussed.

**Part 2**

The second part is the easier one of the two parts to measure. With the sentiment tools all returning either a positive, negative or neutral label, I decide that I will solely be focussing on the positive and negative posts. This means that these are also the posts that will be taken into considerations when looking at volume. The volume will be measured by counting the total of positive and negative posts for a given day. This daily total will then be divided by the average total posts of the last 7 days.

- $\text{Relative volume}_{t0} = \frac{\text{Total positive posts}_{t0}+\text{Total negative posts}_{t0}}{Rolling mean 7(\text{Total positive posts}_{t0}+\text{Total negative posts}_{t0})}$

Calculating the final sentiment is done using multiplying the relative volume times the sentiment.

- $\text{Sentiment}_{t0} = \text{Sentiment (part 1)}_{t0}*\text{Sentiment relative volume (part 2)}_{t0}$


**VADER**

In [2]:
def calc_relative_vol(df):
    df['[s1]rel_vol'] = df['[s1]total'] / df['[s1]total'].rolling(7).mean()
    df['[s2]rel_vol'] = df['[s2]total'] / df['[s2]total'].rolling(7).mean()

    # Dropping NA values
    df.dropna(inplace=True)
    return df

In [3]:
def calc_final_sent(df):
    df['[s1M1]final_sent'] = df['[s1]method_1'] * df['[s1]rel_vol']
    df['[s1M3]final_sent'] = df['[s1]method_3'] * df['[s1]rel_vol']
    df['[s2M1]final_sent'] = df['[s2]method_1'] * df['[s2]rel_vol']
    df['[s2M3]final_sent'] = df['[s2]method_3'] * df['[s2]rel_vol']

    return df

In [5]:
rootdir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\sentiment\VADER"

save = False

if save:
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            # Create csv_path
            csv_path = os.path.join(subdir, file)

            # Read csv
            df = pd.read_csv(csv_path)

            # Calculating relative volume and final sentiment
            df = calc_relative_vol(df)
            df = calc_final_sent(df)


            #Save file
            df.to_csv(csv_path, encoding='utf-8', index=False)

**finBERT**

In [68]:
def calc_relative_vol(df):
    df['[BERT]rel_vol'] = df['[BERT]total'] / df['[BERT]total'].rolling(7).mean()

    # Dropping NA values
    df.dropna(inplace=True)
    return df

In [69]:
def calc_final_sent(df):
    df['[BERT]final_sent'] = df['[BERT]method_1'] * df['[BERT]rel_vol']

    return df

In [71]:
rootdir = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\sentiment\finBERT"

save = False

if save:
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            # Create csv_path
            csv_path = os.path.join(subdir, file)

            # Read csv
            df = pd.read_csv(csv_path)

            # Calculating relative volume and final sentiment
            df = calc_relative_vol(df)
            df = calc_final_sent(df)

            #Save file
            df.to_csv(csv_path, encoding='utf-8', index=False)

## ToDo <a class="anchor" id="ToDo"></a>

[Go back up](#top)

In [67]:
csv_path = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\sentiment\finBERT\m1f1\AMD.csv"
df = pd.read_csv(csv_path)
df = calc_relative_vol(df)
df = calc_final_sent(df)
df

,date,[BERT]pos,[BERT]neg,[BERT]total,[BERT]method_1,[BERT]rel_vol,[BERT]final_sent
6,2018-05-13,4,2,6,0.666667,0.636364,0.424242
7,2018-05-14,2,1,3,0.666667,0.338710,0.225806
8,2018-05-15,3,2,5,0.600000,0.625000,0.375000
9,2018-05-16,10,9,19,0.526316,2.145161,1.129032
10,2018-05-17,12,6,18,0.666667,2.032258,1.354839
...,...,...,...,...,...,...,...
834,2020-08-27,19,25,44,0.431818,1.088339,0.469965
835,2020-08-28,23,15,38,0.605263,0.953405,0.577061
836,2020-08-29,10,3,13,0.769231,0.340824,0.262172
837,2020-08-30,7,2,9,0.777778,0.248031,0.192913


In [252]:
csv_path = r"E:\Users\Christiaan\Large_Files\Thesis\reddit\comments\filtered\filter_1\MU.csv"
comments_df = pd.read_csv(csv_path)

In [253]:
ticker = "MU"
return_df = clean_data(comments_df)

Runtime for calc_sent_finbert: 107.86515927314758 seconds


In [57]:
df['len'] = df['body'].str.len()
df['keep'] = (df['len'] / 512 < 200)

In [13]:
ticker_list = ['AAPL', 'AMD', 'AMZN', 'ATVI', 'BA', 'BABA', 'BAC', 'DIS', 'F', 'GE', 'GME', 'IQ', 'LULU', 'MSFT', 'MU', 'NFLX', 'NVDA', 'SBUX', 'SHOP', 'SNAP', 'SQ', 'TLRY', 'TSLA', 'V', 'WMT']
import time

for ticker in ticker_list:
    print(f"Processing: {ticker.ljust(4)}", end='\r')
    time.sleep(1)


KeyboardInterrupt: 